World Development Indicators - worldbank.org <br/>
Fabio Cardoso - Dec/2021 <br/><br/>

Datasets: <br/><br/>

Country Indicators: <br/>
https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators <br/>
Note: dataset updated with the topics presented in https://data.worldbank.org/indicator?tab=all - Utils notebook. <br/><br/>

Country Codes ISO 3166: <br/>
https://www.iban.com/country-codes <br/><br/>

Note: <br/>
For some orphans indicators, the topic 'private-sector' was assumed - Utils notebook. <br/>
. Firms visited or required meetings with tax officials (% of firms) <br/>
. Net ODA provided to the least developed countries (% of GNI) <br/>
. Net ODA provided, to the least developed countries (current US )−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡2015𝑈𝑆) <br/>
. Net ODA provided, total (current US$)

In [1]:
# Parameters

category_filter = [] #if nothing is defined, no filter will be applied
cloud = True #environment

In [2]:
# Imports 

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import zlib
import pickle
from threading import Thread
from multiprocessing import Pool
import time
from datetime import datetime

In [3]:
# The code was removed by Watson Studio for sharing.

In [6]:
# Storage instantiation

if cloud:
    import types
    import pandas as pd
    import ibm_boto3
    from botocore.client import Config

    def __iter__(self): return 0

    cnx_fcsinsights = ibm_boto3.client(
        service_name = 's3',
        ibm_api_key_id = parm_ibm_api_key_id,
        ibm_service_instance_id = parm_ibm_service_instance_id,
        ibm_auth_endpoint = parm_ibm_auth_endpoint,
        config = Config(signature_version='oauth'),
        endpoint_url = parm_endpoint_url
    )

bkt_in = parm_bkt_in
bkt_out = parm_bkt_out

key_in = 'WDI_plus_category.csv.gz'
key_out = 'WDI treated-step1.csv.gz'

In [7]:
# Read original data

if cloud:
    cnx_fcsinsights.download_file(Bucket=bkt_in, Key=key_in, Filename=key_in)
    df = pd.read_csv('WDI_plus_category.csv.gz', compression='gzip')
else:
    df = pd.read_csv('F:WDI Working Files/WDI_plus_category.csv.gz', compression='gzip')

In [5]:
# Selecting specific category (if some was informed in the parameters cell)

print(df.shape)
if len(category_filter) > 0:
    df = df[df['Category'].isin(category_filter)]
print(df.shape)

(551152, 67)
(551152, 67)


In [7]:
# Unpivoting the dataframe

df = df.drop(columns={'Unnamed: 0','Country Name','Indicator Name','Category'}).melt(id_vars=['Country Code','Indicator Code'], var_name='Year', value_name='Value')

In [8]:
# # of combinations

print("Combinations", df.drop_duplicates(subset=['Country Code','Indicator Code']).shape[0])

Combinations 383838


In [9]:
# Breaking dataframe for parallel processing in posterior steps

part = 0
df['Part'] = 0

df.sort_values(by=['Country Code','Indicator Code','Year'], inplace=True)
df = df.reset_index().drop(columns=['index'])

count = 0
for country_code in df['Country Code'].unique():
    df_country = df[df['Country Code']==country_code]
    for indicator_code in df_country['Indicator Code'].unique():
        
        #Contador
        count +=1
        if count%1000==0:
            print(count, datetime.now())
            
        part +=1
        if part==5:
            part=1
        
        df_4upd = df_country[(df_country['Indicator Code']==indicator_code)]
        ixs = df_4upd.index
        df.at[ixs, 'Part'] = part

1000 2022-01-01 09:39:12.853093
2000 2022-01-01 09:42:57.637714
3000 2022-01-01 09:46:40.079143
4000 2022-01-01 09:50:13.128215
5000 2022-01-01 09:53:49.415493
6000 2022-01-01 09:57:31.770860
7000 2022-01-01 10:01:20.010022
8000 2022-01-01 10:04:53.002025
9000 2022-01-01 10:08:40.830911
10000 2022-01-01 10:12:26.294653
11000 2022-01-01 10:15:58.563994
12000 2022-01-01 10:19:40.676158
13000 2022-01-01 10:23:27.484164
14000 2022-01-01 10:27:29.179529
15000 2022-01-01 10:31:22.447590
16000 2022-01-01 10:35:07.447557
17000 2022-01-01 10:38:45.225124
18000 2022-01-01 10:42:20.922167
19000 2022-01-01 10:45:56.429684
20000 2022-01-01 10:49:30.614998
21000 2022-01-01 10:53:00.444021
22000 2022-01-01 10:56:39.447994
23000 2022-01-01 11:00:29.118896
24000 2022-01-01 11:04:31.918165
25000 2022-01-01 11:08:40.212260
26000 2022-01-01 11:12:42.528807
27000 2022-01-01 11:16:43.016896
28000 2022-01-01 11:20:44.047186
29000 2022-01-01 11:24:56.312543
30000 2022-01-01 11:28:26.982811
31000 2022-01-01 11

In [10]:
# Save intermediate result

if cloud:
    df.to_csv(key_out, compression='gzip', index=False)
    cnx_fcsinsights.upload_file(Bucket=bkt_out, Key=key_out, Filename=key_out)
else:
    df.to_csv('F:/WDI Working Files/'+key_out, compression='gzip', index=False)

In [11]:
print('ok')

ok
